In [2]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, Imputer
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [3]:
filePath = "/home/debargha/Documents/ClimateWatcher/data.csv"

In [4]:
df = pd.read_csv(filePath, engine='python')
pf = df

In [5]:
df.rename(columns={'stn_code': 'station_code', 'location': 'city', 'location_monitoring_station': 'monitoring_station'}, inplace=True)
df.head()

,station_code,sampling_date,state,city,agency,type,so2,no2,rspm,spm,monitoring_station,pm2_5,date
0,150,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",4.8,17.4,NaN,NaN,NaN,NaN,1990-02-01
1,151,February - M021990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,3.1,7.0,NaN,NaN,NaN,NaN,1990-02-01
2,152,February - M021990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.2,28.5,NaN,NaN,NaN,NaN,1990-02-01
3,150,March - M031990,Andhra Pradesh,Hyderabad,NaN,"Residential, Rural and other Areas",6.3,14.7,NaN,NaN,NaN,NaN,1990-03-01
4,151,March - M031990,Andhra Pradesh,Hyderabad,NaN,Industrial Area,4.7,7.5,NaN,NaN,NaN,NaN,1990-03-01


In [1]:
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d') # date parse
df['date'] = df['date'].dt.year # year
df['date'] = df['date'].dt.month #month
df
df['type'].dropna(axis=0, inplace=True)

NameError: name 'pd' is not defined

In [7]:
df.drop(df.columns[[0,1,4,8,9,10,11]], axis=1, inplace=True)

In [9]:
df

,state,city,type,so2,no2,date
0,Andhra Pradesh,Hyderabad,"Residential, Rural and other Areas",4.8,17.4,1990.0
1,Andhra Pradesh,Hyderabad,Industrial Area,3.1,7.0,1990.0
2,Andhra Pradesh,Hyderabad,"Residential, Rural and other Areas",6.2,28.5,1990.0
3,Andhra Pradesh,Hyderabad,"Residential, Rural and other Areas",6.3,14.7,1990.0
4,Andhra Pradesh,Hyderabad,Industrial Area,4.7,7.5,1990.0
5,Andhra Pradesh,Hyderabad,"Residential, Rural and other Areas",6.4,25.7,1990.0
6,Andhra Pradesh,Hyderabad,"Residential, Rural and other Areas",5.4,17.1,1990.0
7,Andhra Pradesh,Hyderabad,Industrial Area,4.7,8.7,1990.0
8,Andhra Pradesh,Hyderabad,"Residential, Rural and other Areas",4.2,23.0,1990.0
9,Andhra Pradesh,Hyderabad,Industrial Area,4.0,8.9,1990.0


In [10]:
df.type.value_counts()

Residential, Rural and other Areas    179014
Industrial Area                        96091
Residential and others                 86791
Industrial Areas                       51747
Sensitive Area                          8980
Sensitive Areas                         5536
RIRUO                                   1304
Sensitive                                495
Industrial                               233
Residential                              158
Name: type, dtype: int64

In [11]:
df.type.replace(['Industrial Area', 'Industrial Areas' ,'Sensitive Areas', 'Sensitive Area'],['Industrial', 'Industrial', 'Sensitive', 'Sensitive'], inplace=True)

In [12]:
df.type.value_counts()

Residential, Rural and other Areas    179014
Industrial                            148071
Residential and others                 86791
Sensitive                              15011
RIRUO                                   1304
Residential                              158
Name: type, dtype: int64

In [13]:
encState = df.iloc[:,0]
encCity = df.iloc[:,1]
encType = df.iloc[:,2]

In [14]:
label_encoder = LabelEncoder()
encState = label_encoder.fit_transform(encState)
encType = label_encoder.fit_transform(encType.astype(str))
encCity = label_encoder.fit_transform(encCity.astype(str))

In [15]:
df['state'] = encState
df['city'] = encCity
df['type'] = encType

In [16]:
df.drop(df.columns[[4]], axis=1, inplace=True)

In [35]:
model = Sequential()
model.add(LSTM(30, return_sequences=True,
               input_shape=(1, 3)))
#model.add(LSTM(input_shape=(200,5),output_dim = 200, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(200))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 1, 30)             4080      
_________________________________________________________________
dropout_13 (Dropout)         (None, 1, 30)             0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 200)               184800    
_________________________________________________________________
dropout_14 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 402       
Total params: 189,282
Trainable params: 189,282
Non-trainable params: 0
_________________________________________________________________


In [39]:
df


,state,city,type,so2,date
0,0,114,4,4.8,1990.0
1,0,114,0,3.1,1990.0
2,0,114,4,6.2,1990.0
3,0,114,4,6.3,1990.0
4,0,114,0,4.7,1990.0
5,0,114,4,6.4,1990.0
6,0,114,4,5.4,1990.0
7,0,114,0,4.7,1990.0
8,0,114,4,4.2,1990.0
9,0,114,0,4.0,1990.0
